In [16]:
import tensorflow as tf
import numpy as np

In [10]:
import seaborn as sns
import pandas as pd
data=pd.read_csv("iris.csv")

In [11]:
#ydata에 대한 원핫 인코딩
from keras.utils import np_utils

In [12]:
#분류
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

In [13]:
import matplotlib.pyplot as plt
epochs=range(1, len(history.history['acc'])+1)
epochs
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title("cost of model")
plt.ylabel("cost")
plt.xlabel("epoch")
plt.legend(['train', 'val'], loc="upper left")
plt.show
print("테스트 정확도:%.4f" %(model.evaluate(xtest, ytest)[1]))

NameError: name 'history' is not defined

In [17]:
#문서간 유사
#문서의 단어들을 수치화 => TDM, Word2Vec
#문서간의 단어들의 차이를 비교 => 유클리디안, 코사인유사도


In [18]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

from numpy import dot
from numpy.linalg import norm

In [19]:
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

In [20]:
print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3))# 1로 2번과 3번이 가장 가까움

0.6666666666666667
0.6666666666666667
1.0000000000000002


In [21]:
data=pd.read_csv("movies_metadata.csv")
data.head(2)
data.shape #(45466,24)

C:\Users\B-7\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(45466, 24)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words="english")

In [23]:
data=data.head(30000)

In [24]:
# data.info()
data['overview']=data['overview'].fillna('') 

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
adult                    30000 non-null object
belongs_to_collection    3354 non-null object
budget                   30000 non-null object
genres                   30000 non-null object
homepage                 4870 non-null object
id                       30000 non-null object
imdb_id                  29988 non-null object
original_language        29997 non-null object
original_title           30000 non-null object
overview                 30000 non-null object
popularity               29996 non-null object
poster_path              29837 non-null object
production_companies     29998 non-null object
production_countries     29998 non-null object
release_date             29969 non-null object
revenue                  29996 non-null float64
runtime                  29919 non-null float64
spoken_languages         29996 non-null object
status                   29955 non-null objec

In [26]:
tfidf_matrix=tfidf.fit_transform(data['overview'])
tfidf_matrix.shape

(30000, 58562)

In [27]:
from sklearn.metrics.pairwise import linear_kernel
cos_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [58]:
cos_sim

array([[1.        , 0.01561351, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01561351, 1.        , 0.0486754 , ..., 0.        , 0.        ,
        0.02282596],
       [0.        , 0.0486754 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.01140332],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.02282596, 0.        , ..., 0.01140332, 0.        ,
        1.        ]])

In [28]:
cos_sim.shape

(30000, 30000)

In [29]:
# data.index
# data['title']

res=pd.Series(data.index, index=data['title']).drop_duplicates() #중복데이터 해결
res.head()
res.tail()


title
Seventeen Again                      29995
Sweet Sixteen                        29996
The Disappearance of Garcia Lorca    29997
The Dramatics: A Comedy              29998
Up the Creek                         29999
dtype: int64

In [30]:
res['Up the Creek']

29999

In [31]:
def get_recommender(title, cos_sim=cos_sim): #cos_sim=linear_kernel(tfidf_ma...)
    idx=res[title]
    sim_scores=list(enumerate(cos_sim[idx]))
    sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True)
    sim_scores=sim_scores[1:11]
    movie_res=[i[0] for i in sim_scores]
    return data['title'].iloc[movie_res]
#     #return sim_scores
print(get_recommender('Toy Story'))

15348                    Toy Story 3
2997                     Toy Story 2
10301         The 40 Year Old Virgin
24523                      Small Fry
23843    Andy Hardy's Blonde Trouble
29202                     Hot Splash
8327                       The Champ
27206     Life Begins for Andy Hardy
1071           Rebel Without a Cause
26304         You're Only Young Once
Name: title, dtype: object


문제 4)vote average가 9점 이상인 영화 5편 추출

In [55]:
def get_recommender_voteAverage(title, cos_sim=cos_sim, size=5): 
    idx=res[title]
    
    sim_scores=list(enumerate(cos_sim[idx]))
    sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    cnt=0
    iterator=1
    saveList=[]
    while cnt<size:
        if data['vote_average'].iloc[sim_scores[iterator][0]]>=9.0:
            saveList.append(iterator)
            cnt+=1
        iterator+=1
    
    movies_res=[sim_scores[i][0] for i in saveList]
    return data['title'].iloc[movies_res]
    #return sim_scores
    
print(get_recommender_voteAverage('Rambo'))

7541                                        A Better Place
20504    Living in Emergency: Stories of Doctors Withou...
18629                                        The Civil War
27769                                                  Pan
27173                                      Willow and Wind
Name: title, dtype: object


문제 5) 추천 영화들의 평균 투표수은?

In [56]:
def get_recommender_voteAverage(title, cos_sim=cos_sim, size=10): 
    idx=res[title]
    
    sim_scores=list(enumerate(cos_sim[idx]))
    sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    cnt=0
    iterator=1
    saveList=[]
    while cnt<size:
        if data['vote_average'].iloc[sim_scores[iterator][0]]>=9.0:
            saveList.append(iterator)
            cnt+=1
        iterator+=1
    
    movies_res=[sim_scores[i][0] for i in saveList]
    return movies_res
    
def get_recommender_countAverage(title, size=10): 
    indexList=get_recommender_voteAverage(title)
    sum=0
    for i in indexList:
        sum+=data['vote_count'].iloc[i]
    return sum/size

In [57]:
print("평균 9점 이상 영화 중 A Better Place와 overview가 유사한 영화 10편의 평균 투표갯수:", get_recommender_countAverage('A Better Place'))

평균 9점 이상 영화 중 A Better Place와 overview가 유사한 영화 10편의 평균 투표갯수: 2.3
